# 이 노트의 목표
---
1. 3/08, 3/11 노트 정리
    











2. ?




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import

import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from PIL import Image


In [28]:
# dir

database_dir = '/content/drive/MyDrive/aiffel/AIFFELthon/1_data/ICBHI_final_database/'
train_test_dir = '/content/drive/MyDrive/aiffel/AIFFELthon/1_data/ICBHI_challenge_train_test.txt'


### 함수1 : get_dataframe

1. return1 : wav, txt 파일명 반환하는 튜플리스트 -> [ (wav, txt), ...  ]
2. return2 : 오디오파일 df 생성

|idx|filename|start|end|crackles|wheezes|duration|train-test|
|---|---|---|---|---|---|---|---|
|0|||||||||
|1|||||||||
|2|||||||||




In [29]:
# dir내의 파일을 이름으로 같이 묶어서 반환하는 함수 -> [ (wav, txt), ( ... ), ... ]

def get_dataframe(database_dir, train_test_dir, sr = 8000):
    '''
    database_dir : raw database (include wav, txt files)
    train_test_dir: train, test 정보 포함된 txt 경로
    '''
    # 디렉토리 내의 모든 파일 목록을 가져옵니다.
    all_files = os.listdir(database_dir)

    wav_files = [file for file in all_files if file.endswith('.wav')]
    txt_files = [file for file in all_files if file.endswith('.txt')]

    # 파일명(확장자 제외)을 기준으로 .wav와 .txt 파일이 모두 있는 경우를 찾아서 짝지어 줍니다.
    wav_txt_name = []
    for wav_file in wav_files:
        file_name_without_extension = os.path.splitext(wav_file)[0] # 확장자를 제외한 파일명

        matching_txt_file = file_name_without_extension + '.txt' # 해당 파일명을 가진 .txt 파일이 있는지 확인
        if matching_txt_file in txt_files:
            wav_txt_name.append((wav_file, matching_txt_file)) # : wav_txt_name = [(filename.wav, filename.txt), ... ]
            # wav_txt_name -> "return1"

    # wav filename만 불러오고, duration 계산
    duration_ls = []
    wav_filename_ls = []
    for i, _ in wav_txt_name:          # wav 파일 이름만 가져옴
        wav_filename_ls.append(i)

        wav_path = database_dir + i
        y, sr = librosa.load(wav_path, sr=sr)
        duration = librosa.get_duration(y=y, sr=sr)
        duration_ls.append(duration)

    # ICBHI_challenge_train_test.txt 파일을 읽어서 DataFrame에 추가할 정보를 추출
    train_test_ls = []
    with open(train_test_dir, 'r') as file:
        for line in file:
            parts = line.strip().split('\t')
            filename = parts[0] + '.wav'  # WAV 파일 이름으로 변환
            train_test = parts[1]  # train 또는 test 값
            train_test_ls.append({'WAV Filename': filename, 'Train_Test': train_test})

    # get dataframe(1) - filename, duration
    wav_duration_df = pd.DataFrame({
        'WAV Filename': wav_filename_ls,
        'Duration': duration_ls
    })

    # ICBHI_challenge_train_test.txt 파일을 읽어서 DataFrame에 추가할 정보를 추출
    train_test_ls = []
    with open(train_test_dir, 'r') as file:
        for line in file:
            parts = line.strip().split('\t')
            filename = parts[0] + '.wav'  # WAV 파일 이름으로 변환
            train_test = parts[1]  # train 또는 test 값
            train_test_ls.append({'WAV Filename': filename, 'Train_Test': train_test})
    train_test_df = pd.DataFrame(train_test_ls)

    # get dataframe(2) : 원래 DataFrame과 새로운 DataFrame을 'WAV Filename'을 기준으로 병합 -> "return2"
    merged_df = pd.merge(wav_dur_df, train_test_df, on='WAV Filename', how='left')

    return wav_txt_name, merged_df


In [30]:
# test
wav_txt_name, merged_df = get_dataframe(database_dir, train_test_dir, sr = 8000)

In [31]:
print(type(wav_txt_name), type(merged_df))
print(len(wav_txt_name))
merged_df.head()

<class 'list'> <class 'pandas.core.frame.DataFrame'>
920


,WAV Filename,Duration,Train_Test
0,101_1b1_Al_sc_Meditron.wav,20.000000,test
1,101_1b1_Pr_sc_Meditron.wav,20.000000,test
2,102_1b1_Ar_sc_Meditron.wav,20.000000,test
3,103_2b2_Ar_mc_LittC2SE.wav,20.000000,train
4,104_1b1_Al_sc_Litt3200.wav,15.856009,test


### 함수2 : 하나의 wav file에 대한 anno df 반환

['start', 'end', 'crackle', 'wheeze']

In [27]:
# 하나의 annotation dataframe
txt_path = database_dir + wav_txt_name[0][1]

tmp_df = pd.read_csv(txt_path,
               sep='\t',
               names = ['start', 'end', 'crackle', 'wheeze'])
tmp_df.head()

,start,end,crackle,wheeze
0,0.036,0.579,0,0
1,0.579,2.450,0,0
2,2.450,3.893,0,0
3,3.893,5.793,0,0
4,5.793,7.521,0,0


In [47]:
# merged_df idx정보로 annotation df 생성

def annot_df(idx, merged_df, database_dir):
    filename = merged_df['WAV Filename'][idx].replace('.wav', '.txt')
    txt_path = database_dir + filename
    annot_df = pd.read_csv(txt_path,
               sep='\t',
               names = ['start', 'end', 'crackle', 'wheeze'])
    annot_df.insert(0, 'WAV Filename', value=merged_df['WAV Filename'][idx])
    return annot_df


In [48]:
annot_df(0, merged_df, database_dir)


,WAV Filename,start,end,crackle,wheeze
0,101_1b1_Al_sc_Meditron.wav,0.036,0.579,0,0
1,101_1b1_Al_sc_Meditron.wav,0.579,2.450,0,0
2,101_1b1_Al_sc_Meditron.wav,2.450,3.893,0,0
3,101_1b1_Al_sc_Meditron.wav,3.893,5.793,0,0
4,101_1b1_Al_sc_Meditron.wav,5.793,7.521,0,0
5,101_1b1_Al_sc_Meditron.wav,7.521,9.279,0,0
6,101_1b1_Al_sc_Meditron.wav,9.279,11.150,0,0
7,101_1b1_Al_sc_Meditron.wav,11.150,13.036,0,0
8,101_1b1_Al_sc_Meditron.wav,13.036,14.721,0,0
9,101_1b1_Al_sc_Meditron.wav,14.721,16.707,0,0


In [ ]:
# train-test df split

def split_train_test_df(merged_df):
    train_df = merged_df[merged_df['train_test'] == 'train']
    test_df = merged_df[merged_df['train_test'] == 'test']
    return train_df, test_df

### 함수3 : audio2mel_save

In [ ]:
# convert_audio_to_melspectrogram

def convert_audio_to_melspectrogram(audio_path, save_path, size=(224, 224)):
    '''
    args:
        audio_path : 변환할 오디오 파일 경로
        save_path : 이미지 저장할 경로
        size : 변환할 이미지 pixel사이즈

    오디오 파일 경로를 받아서 저장 경로에 (224, 224, 3) 크기의 filename_mSpec.jpg 저장
    0. 변수 지정
    1. 파일이름, 디렉토리 정의
    2. audio -> mel
    3. mel -> image(jpg) -> resizing
    4. save image
    '''
    # 0. mel 변수 지정 -> 함수 매개변수로 옮겨도 되는 부분
    sample_rate = 8000  # 샘플링 레이트 (or 16000)
    n_mels = 64  # 멜 필터뱅크의 수
    nfft = 512  # FFT 윈도우 크기
    hop_length = nfft // 2  # 프레임 사이의 샘플 수
    f_max = 2000  # 최대 주파수

    # 1. .wav -> .jpg
    # .wav 파일 경로에서 디렉터리와 파일명(확장자 포함) 분리
    dir_path, filename = os.path.split(audio_path)
    # 파일명에서 확장자를 제거하고, .jpg 확장자를 추가
    base_filename = os.path.splitext(filename)[0]
    jpg_filename = f"{base_filename}_mSpec.jpg"
    # 디렉터리 내에 .jpg 파일 경로 생성
    save_path += jpg_filename


    # 2. to mel
    y, sr = librosa.load(audio_path, sr=sample_rate)
    S = librosa.feature.melspectrogram(y=y,
                                       sr=sr,
                                       n_fft=nfft,
                                       hop_length=hop_length,
                                       n_mels=n_mels,
                                       fmax=f_max)
    S_dB = librosa.power_to_db(S, ref=np.max)


    # 3. to img
    plt.figure(figsize=(2.24, 2.24))  # 이미지 크기를 인치 단위로 설정 (224x224 픽셀이 되도록 DPI 조정 필요)
    ax = plt.axes([0,0,1,1], frameon=False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.autoscale(tight=True)
    librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel')

    # 이미지로 저장하기 전에 buffer에 저장
    plt.savefig(save_path, dpi=100)  # DPI를 조정하여 대략적인 픽셀 크기 맞춤
    #plt.savefig(save_path, bbox_inches='tight', pad_inches=0, dpi=100) # DPI를 조정하여 대략적인 픽셀 크기 맞춤
    plt.close()
    # 이미지를 다시 열어서 224x224x3으로 조정
    img = Image.open(save_path).convert('RGB')
    img = img.resize(size, Image.LANCZOS)


    # 4. save img
    if os.path.exists(save_path):
        pass
    else:
        img.save(save_path)

print('ㅎ')

In [ ]:
# 사용

save_path = '/content/drive/MyDrive/aiffel/AIFFELthon/1_data/224mel_database240311/'

for audio_file_name in wav_filename_ls:
    audio_path = data_root_dir + audio_file_name

    convert_audio_to_melspectrogram(audio_path, save_path, size=(224,224)) # 사용할때 주석 제거
    break

### 비율 고민


In [ ]:
#

pix_size = 224
maxHz = 2000 # 이부분은 mel 변환 고려해서 저장
duration = duration # 각 오디오 파일 길이

crackle_minHz = 100
crackle_maxHz = 200
wheeze_minHz = 100
wheeze_maxHz = 2000

'''
start, end -> column name

'''

# 비율 (224중 얼마 의 느낌으로 나옴)
x_start = (start / duration) * pix_size
x_end = (end / duration) * pix_size

y_start = (1 - (crackle_maxHz / maxHz)) * pix_size
y_end = (1 - (crackle_minHz / maxHz)) * pix_size

print(x_start, x_end, y_start, y_end) # 21.672 46.872 201.6 212.79999999999998


# 비율 (0 ~ 1)사이 값으로 출력 고민